# NYC buildings dataset with zoom: GeoPandas and SpatialPandas

Data from http://s3.amazonaws.com/datashader-data/nyc_buildings.parq.zip

Convert to GeoPandas using something like:
```
sdf = spatialpandas.io.read_parquet(input_directory)
gdf = sdf.to_geopandas()
gdf.to_parquet(output_filename)
```

In [ ]:
import colorcet as cc
import datashader as ds
import geopandas
import numpy as np
import spatialpandas.io

In [ ]:
spd_dir = '/Users/iant/data/hv/nyc_buildings.parq'
gpd_file = '/Users/iant/data/hv/gpd.parq'
zoom_factor = 30

In [ ]:
# Load data into GeoPandas and SpatialPandas GeoDataFrames
sdf = spatialpandas.io.read_parquet(spd_dir)
sdf.build_sindex()  # IMPORTANT: Explicitly build SpatialPandas spatial index or zoom will be very slow
gdf = geopandas.read_parquet(gpd_file)

In [ ]:
# Bounds using zoom_level
def get_bounds(zoom_factor):
    x_range = (-8277717.335387823, -8201084.997925733)
    y_range = (4935946.1063207975, 5006441.987955982)
    if zoom_factor == 1:
        return x_range, y_range

    xmid = np.mean(x_range)
    dx = np.diff(x_range)[0]
    ymid = np.mean(y_range)
    dy = np.diff(y_range)[0]

    dx /= zoom_factor
    dy /= zoom_factor
    return (xmid, xmid + dx), (ymid, ymid + dy)

x_bounds, y_bounds = get_bounds(zoom_factor)

## Display both outputs

In [ ]:
# GeoPandas
canvas = ds.Canvas(plot_height=1000, plot_width=1000, x_range=x_bounds, y_range=y_bounds)
agg = canvas.polygons(source=gdf, geometry="geometry", agg=ds.count())
ds.transfer_functions.shade(agg, cmap=cc.bgy, how="eq_hist")

In [ ]:
# SpatialPandas
agg = canvas.polygons(source=sdf, geometry="geometry", agg=ds.count())
ds.transfer_functions.shade(agg, cmap=cc.bgy, how="eq_hist")

## Time GeoPandas

In [ ]:
%%timeit
agg = canvas.polygons(source=gdf, geometry="geometry", agg=ds.count())

## Time SpatialPandas

In [ ]:
%%timeit
agg = canvas.polygons(source=sdf, geometry="geometry", agg=ds.count())